In [3]:
!pip install qdrant-client
!pip install sentence_transformers

  Using cached qdrant_client-1.11.1-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio_tools-1.66.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached grpcio-1.66.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached qdrant_client-1.11.1-py3-none-any.whl (259 kB)
Using cached grpcio_tools-1.66.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
Using cached grpcio-1.66.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.7 MB)
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 6.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.3


In [4]:
!pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
import json
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()



/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [7]:
with open('diabetes_data_with_vectors', 'r') as f_in:
    diabetes_data_with_vectors = json.load(f_in)

In [14]:
api_key = os.getenv('QDRANT_API_KEY')


client = QdrantClient(
    url="https://8999b86c-f8b2-4d60-bdfa-8c68d39daae7.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key=api_key,timeout=200
)

print(api_key)

r09oP5L3SaVcMi_MvBC3yBOuRpcYChNeZjzTiEHr5cyoyHiP00CyEw


In [15]:
client.delete_collection(
    collection_name="diabetes")

True

In [16]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="diabetes",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [17]:
from qdrant_client.http.models import PointStruct



# Upsert data in chunks
#for chunk in chunk_data(diabetes_data_with_vectors, chunk_size):
client.upsert(
    collection_name="diabetes",
    points=[
        PointStruct(
            id=idx,
            vector=data['qa_text_embeddings'],
            payload={"question": data['question'], "answer": data['answer'],"id": data['id'], "rand_number": idx % 10}
        )
        for idx, data in enumerate(diabetes_data_with_vectors)
    ]
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')


In [19]:
query_vector = embedding_model.encode('Man utd and chelsea updates').tolist()

In [20]:
hits = client.search(
    collection_name="diabetes",
    query_vector=query_vector,
    limit=5  # Return 5 closest points
)

In [21]:
collection_name = "diabetes"
count = client.count(collection_name=collection_name)

print(f"Total number of points in the collection '{collection_name}': {count}")

Total number of points in the collection 'diabetes': count=1071


In [22]:
for hit in hits:
    print(hit.score)
    break

0.14268471
